In [2]:
!git clone https://github.com/InterxPim/Models.git

Cloning into 'Models'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [4]:
%cd Models/Drépanocytose

/content/Models


In [ ]:
!pip install -r requirement.txt

In [ ]:
import tensorflow as tf
import keras as K
tf.__version__, K.__version__

In [ ]:
import source.toolkit as tk
import source.dojo_tools as dj

from source.toolkit import list_channels
from source.toolkit import list_channels_df

# load main class object for monitoring blood cells
from source.SickleML_Monitor import CountAdheredBloodCells

# loading tools for extracting gdrive data
import source.load_data_tools as loading_tools

import os
import pandas as pd
import time
from tqdm import tqdm
import sys
import gdown
from natsort import natsorted

In [ ]:
os.makedirs(f'./Phase2_Pretrained-models/Resnet50/', exist_ok = True)
os.chdir(f'./Phase2_Pretrained-models/Resnet50/') 

loading_tools.resnet50_gdrive_()
    
os.chdir(f'../')
os.chdir(f'../')


os.makedirs(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/', exist_ok = True)
os.chdir(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/') 

loading_tools.ce_jaccard_gdrive_()
    
os.chdir(f'../')
os.chdir(f'../')


In [ ]:
channel_path= './data/Manual-VS-AI/'
os.makedirs(channel_path, exist_ok=True)
os.chdir(channel_path)
loading_tools.laminin_channel_gdrive_()
os.chdir('../')
os.chdir('../')
channel_filenames  =  os.listdir(channel_path)

In [ ]:
print(f'Number of channels: {len(channel_filenames)}')
channel_filenames

In [ ]:
def load_ensembles(Phase1_path_model, Phase2_path_model):

    Phase1_path = './Phase1_Pretrained-models/' + Phase1_path_model + '/'# folder for Phase I 
    Phase1_filenames = natsorted(os.listdir(Phase1_path))
    Phase1_final_filenames = []
    for ii in range(len(Phase1_filenames)):
        if ii % 2 != 0:
            Phase1_final_filenames.append(Phase1_filenames[ii].replace('.json', ''))

    Phase2_path = './Phase2_Pretrained-models/' + Phase2_path_model + '/'# folder for Phase II
    Phase2_filenames = natsorted(os.listdir(Phase2_path))
    Phase2_final_filenames = []
    for ii in range(len(Phase2_filenames)):
        if ii % 2 != 0:
            Phase2_final_filenames.append(Phase2_filenames[ii].replace('.json',''))


    Phase1_ensemble = tk.load_zoo(Phase1_path, Phase1_final_filenames) # loading the Phase I ensemble (expect: 7)
    Phase2_ensemble = tk.load_zoo(Phase2_path, Phase2_final_filenames) # loading the Phase I ensemble (expect: 5)
    return Phase1_ensemble, Phase2_ensemble

def create_final_df(counts, times):
    counts_df = pd.DataFrame(counts)
    counts_df.columns = ['filename', 'def-sRBC', 'nondef-sRBC', 'Other']
    times_df = pd.DataFrame(times)
    times_df.columns = ['time_secs']
    final_df = pd.concat([counts_df, times_df], axis = 1)
    return final_df

In [ ]:
%%time 



counts, times = [], [] 
count_container, time_container = [], []

rbc_thres = [0.4]
wbc_thres = [0.4]
other_thres = [0.9]

Phase1_names, Phase2_names = 'ce-jaccard_encoder-decoder-net', 'Resnet50'

Phase1_ensemble, Phase2_ensemble = load_ensembles(Phase1_names, Phase2_names)
counts, times = [], [] 

for index, filenames in enumerate(channel_filenames):
    for rep in ((".png", ""), (".jpg", "")):
        clean_filename = filenames.replace(*rep)
    print('Analysis:', index, '| Channel:', clean_filename)
    print('==================================================================')
    channel = CountAdheredBloodCells(channel_path, filenames) # calling the class object
    # calling the function to output cell counts
    start = time.time()
    sRBC, WBC, Others, img_container, sRBC_container, WBC_container, Other_container = channel.call_pipeline(Phase1_ensemble, Phase2_ensemble, rbc_thres, wbc_thres, other_thres)
    end = time.time()
    run_time = end-start
            
    times.append([run_time])
    counts.append([filenames, sRBC, WBC, Others])
            
    final_df = create_final_df(counts,times)
   # final_df.to_csv(f'./AI-vs-Human_counts/{Phase1_name}_{Phase2_name}.csv', index = False)


In [ ]:
print("Here is the predicted counts ...")
final_df